In [68]:
# imports
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [69]:
ruta = "/content/train.csv"
df = pd.read_csv(ruta)

In [70]:
print(df.shape)
df.head()

(732, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,51.0,1.0,1.0,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0
1,54.0,1.0,3.0,120.0,237.0,0.0,0.0,150.0,1.0,1.5,-9.0,-9.0,7.0,2
2,63.0,1.0,4.0,140,0,?,2.0,149,0,2,1,?,?,2
3,52.0,0.0,2.0,140.0,-9.0,0.0,0.0,140.0,0.0,0.0,-9.0,-9.0,-9.0,0
4,55.0,1.0,4.0,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       732 non-null    float64
 1   sex       732 non-null    float64
 2   cp        732 non-null    float64
 3   trestbps  732 non-null    object 
 4   chol      732 non-null    object 
 5   fbs       732 non-null    object 
 6   restecg   732 non-null    float64
 7   thalach   732 non-null    object 
 8   exang     732 non-null    object 
 9   oldpeak   732 non-null    object 
 10  slope     732 non-null    object 
 11  ca        732 non-null    object 
 12  thal      732 non-null    object 
 13  label     732 non-null    int64  
dtypes: float64(4), int64(1), object(9)
memory usage: 80.2+ KB


# 1. Limpieza del dataset de entrenamiento

En esta primera fase, procederemos a limpiar el dataset siguiendo las situaciones más comunes.

## 4.1 Datos faltantes

En primer lugar estudiaremos la existencia de registros incompletos en nuestro dataset.

In [72]:
# Ver valores nulos en el DataFrame
valores_nulos = df.isnull().sum()

# Imprimir el número de valores nulos en cada columna
print(valores_nulos)

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
label       0
dtype: int64


Observamos que no hay registros vacíos en el dataset de entrenamiento, por lo que procedemos al siguiente paso.

En segundo lugar, estudiaremos el reparto de valores de cada variable buscando posibles anomalías.

Empezamos con la variable de la edad.

In [73]:
# Obtener la distribución de valores de la variable "age" y ordenar por edad ascendente
reparto_age = df['age'].value_counts().sort_index()

# Imprimir el reparto de valores ordenado por edad ascendente
print(reparto_age)


age
28.0     1
29.0     2
30.0     1
31.0     2
32.0     3
33.0     1
34.0     5
35.0     8
36.0     6
37.0    10
38.0    12
39.0    13
40.0     7
41.0    24
42.0    15
43.0    19
44.0    15
45.0    14
46.0    20
47.0    13
48.0    23
49.0    22
50.0    21
51.0    27
52.0    29
53.0    27
54.0    38
55.0    29
56.0    34
57.0    33
58.0    36
59.0    29
60.0    24
61.0    25
62.0    26
63.0    24
64.0    17
65.0    14
66.0    12
67.0     9
68.0     9
69.0    12
70.0     5
71.0     4
72.0     2
73.0     1
74.0     4
75.0     3
76.0     2
Name: count, dtype: int64


Como la edad es un número entero, nos podemos ahorrar sus decimales, por lo que pasamos la variable a formato entero.

In [74]:
# Convertir los valores de la columna 'age' a enteros en el DataFrame df
df['age'] = df['age'].astype(int)

# Obtener la distribución de valores y ordenarla por edad ascendente
reparto_age = df['age'].value_counts().sort_index()

# Imprimir el reparto de valores ordenado por edad ascendente
print(reparto_age)


age
28     1
29     2
30     1
31     2
32     3
33     1
34     5
35     8
36     6
37    10
38    12
39    13
40     7
41    24
42    15
43    19
44    15
45    14
46    20
47    13
48    23
49    22
50    21
51    27
52    29
53    27
54    38
55    29
56    34
57    33
58    36
59    29
60    24
61    25
62    26
63    24
64    17
65    14
66    12
67     9
68     9
69    12
70     5
71     4
72     2
73     1
74     4
75     3
76     2
Name: count, dtype: int64


Ahora procedemos a estudiar la variable del sexo.

In [75]:
# Obtener la distribución de valores de la variable "sex" y ordenar por edad ascendente
reparto_sex = df['sex'].value_counts().sort_index()

# Imprimir el reparto de valores ordenado por edad ascendente
print(reparto_sex)

sex
0.0    154
1.0    578
Name: count, dtype: int64


Vemos que el dataset otorga el valor 1 al sexo masculino, y 0 al femenino, por lo que podemos descartar los decimales también.

In [76]:
# Convertir los valores de la columna 'sex' a enteros en el DataFrame df
df['sex'] = df['sex'].astype(int)

# Obtener la distribución de valores y ordenarla por sexo ascendente
reparto_sex = df['sex'].value_counts().sort_index()

# Imprimir el reparto de valores ordenado por sexo ascendente
print(reparto_sex)


sex
0    154
1    578
Name: count, dtype: int64


Procedemos a estudiar la variable cp, que otorga valores del 1 al 4 según el tipo de dolor del paciente.

In [77]:
# Obtener la distribución de valores de la variable "cp" y ordenar por edad ascendente
reparto_cp = df['cp'].value_counts().sort_index()

# Imprimir el reparto de valores ordenado por edad ascendente
print(reparto_cp)

cp
1.0     35
2.0    138
3.0    168
4.0    391
Name: count, dtype: int64


Descartamos los decimales de la misma manera.

In [78]:
# Convertir los valores de la columna 'cp' a enteros en el DataFrame df
df['cp'] = df['cp'].astype(int)

# Obtener la distribución de valores y ordenarla por edad ascendente
reparto_cp = df['cp'].value_counts().sort_index()

# Imprimir el reparto de valores ordenado por cp ascendente
print(reparto_cp)


cp
1     35
2    138
3    168
4    391
Name: count, dtype: int64


Procedemos con la variable trestbps, que corresponde a la presión arterial en reposo.

In [79]:
# Obtener la distribución de valores de la variable "trestbps" y ordenar por edad ascendente
reparto_trestbps = df['trestbps'].value_counts().sort_index()

# Imprimir el reparto de valores ordenado por edad ascendente
print(reparto_trestbps)

trestbps
0         1
100       5
100.0     9
102.0     2
104       2
         ..
94.0      2
95        3
96        1
98.0      1
?        47
Name: count, Length: 95, dtype: int64


En primer lugar, al no existir ningún registro decimal, los descartamos.

In [80]:
# Eliminar los decimales de la columna 'trestbps'
df['trestbps'] = df['trestbps'].astype(str).str.split('.').str[0]

# Obtener la distribución de valores y ordenarla por trestbps ascendente
reparto_trestbps = df['trestbps'].value_counts().sort_index()

# Imprimir el reparto de valores ordenado por trestbps ascendente
print(reparto_trestbps)


trestbps
0        1
100     14
102      2
104      3
105      7
106      2
108      6
110     44
112     12
113      1
114      2
115     14
116      2
117      1
118      9
120    109
122     11
123      1
124     11
125     24
126      6
127      1
128     15
130     88
132     10
134      9
135     17
136      7
138      9
140     78
142      7
144      3
145     11
146      3
148      2
150     42
152      4
154      2
155      6
158      2
160     41
164      1
165      2
170     12
172      2
174      1
178      3
180     11
185      1
190      2
192      1
200      2
92       1
94       2
95       3
96       1
98       1
?       47
Name: count, dtype: int64


Vemos que hay un outlier, ya que hay un registro con valor a 0, lo que no tiene sentido en nuestro dataset. Al solo ser uno, podemos proceder a descartarlo.

In [81]:
# Eliminar los decimales de la columna 'trestbps'
df['trestbps'] = df['trestbps'].astype(str).str.split('.').str[0]

# Filtrar los valores diferentes de '0'
df = df[df['trestbps'] != '0']

# Obtener la distribución de valores y ordenarla por 'trestbps' ascendente
reparto_trestbps = df['trestbps'].value_counts().sort_index()

# Imprimir el reparto de valores ordenado por 'trestbps' ascendente
print(reparto_trestbps)


trestbps
100     14
102      2
104      3
105      7
106      2
108      6
110     44
112     12
113      1
114      2
115     14
116      2
117      1
118      9
120    109
122     11
123      1
124     11
125     24
126      6
127      1
128     15
130     88
132     10
134      9
135     17
136      7
138      9
140     78
142      7
144      3
145     11
146      3
148      2
150     42
152      4
154      2
155      6
158      2
160     41
164      1
165      2
170     12
172      2
174      1
178      3
180     11
185      1
190      2
192      1
200      2
92       1
94       2
95       3
96       1
98       1
?       47
Name: count, dtype: int64


Vemos también que existen 47 valores desconocidos con valor '?'. Viendo la distribución de valores, vemos que la mayoría de los valores se encuentran entre 120 y 130. Vamos a calcular la media por si acaso.

In [82]:
# Filtrar los valores diferentes de '?' en la columna 'trestbps'
df_filtered = df[df['trestbps'] != '?']

# Convertir la columna 'trestbps' a tipo entero
df_filtered['trestbps'] = df_filtered['trestbps'].astype(int)

# Calcular la media de los valores en la columna 'trestbps'
media_trestbps = df_filtered['trestbps'].mean()

# Imprimir la media
print("Media de trestbps", media_trestbps)


Media de trestbps 132.16812865497076


<ipython-input-82-76c34d7b955a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['trestbps'] = df_filtered['trestbps'].astype(int)


Al ser la media 130, que coincide con el rango de la moda, decidimos darle el valor 130 a los 47 registros desconocidos en esta fase inicial.

In [83]:
# Reemplazar los valores '?' por 130 en la columna 'trestbps'
df['trestbps'].replace('?', 130, inplace=True)

# Convertir la columna 'trestbps' a tipo numérico
df['trestbps'] = pd.to_numeric(df['trestbps'])

# Obtener la distribución de valores y ordenarla por 'trestbps' ascendente
reparto_trestbps = df['trestbps'].value_counts().sort_index()

# Imprimir el reparto de valores ordenado por 'trestbps' ascendente
print(reparto_trestbps)


trestbps
92       1
94       2
95       3
96       1
98       1
100     14
102      2
104      3
105      7
106      2
108      6
110     44
112     12
113      1
114      2
115     14
116      2
117      1
118      9
120    109
122     11
123      1
124     11
125     24
126      6
127      1
128     15
130    135
132     10
134      9
135     17
136      7
138      9
140     78
142      7
144      3
145     11
146      3
148      2
150     42
152      4
154      2
155      6
158      2
160     41
164      1
165      2
170     12
172      2
174      1
178      3
180     11
185      1
190      2
192      1
200      2
Name: count, dtype: int64


Ahora ya tendríamos bien normalizada esta variable.

A continuación procedemos a estudiar la variable *chol*, que corresponde con el colesterol sérico en sangre, del cual su disposición es la siguiente:

In [84]:
# Obtener la distribución de valores de la variable 'chol'
distribucion_chol = df['chol'].value_counts().sort_index()

# Imprimir la distribución de valores de 'chol'
print(distribucion_chol)


chol
-9.0      16
0        134
100        1
100.0      1
117.0      1
        ... 
529.0      1
564.0      1
603.0      1
85.0       1
?          5
Name: count, Length: 280, dtype: int64


De la misma manera, al no existir rango decimal lo convertimos a enteros. Tampoco encontramos ningún outlier.

In [85]:
# Convertir los valores numéricos de la columna 'chol' a enteros
df['chol'] = pd.to_numeric(df['chol'], errors='coerce').astype('Int64')

# Obtener la distribución de valores de la variable 'chol' y ordenarla por los valores de colesterol de forma ascendente
distribucion_chol = df['chol'].value_counts().sort_index()

# Imprimir la distribución de valores de 'chol'
print(distribucion_chol)

chol
-9      16
0      134
85       1
100      2
117      1
      ... 
491      1
518      1
529      1
564      1
603      1
Name: count, Length: 204, dtype: Int64


Vemos ahora qué hacer con los 16 registros evaluados a -9, ya que no tiene sentido en la evaluación del colesterol.

In [86]:
# Calcular la media del campo 'chol'
media_chol = df['chol'].mean()

# Calcular la moda del campo 'chol'
moda_chol = df['chol'].mode()[0]

# Imprimir la media y la moda del campo 'chol'
print("Media de chol:", media_chol)
print("Moda de chol:", moda_chol)


Media de chol: 195.6267217630854
Moda de chol: 0


En este caso la media no es lo más óptimo, ya que lo común como vemos en la moda es no tener colesterol en sangre, por lo que sustituiremos los valores por la moda.

In [87]:
# Reemplazar los valores -9 por 0 en la columna 'chol'
df['chol'].replace(-9, 0, inplace=True)

# Obtener la distribución de valores de la variable 'chol' y ordenarla por los valores de colesterol de forma ascendente
distribucion_chol = df['chol'].value_counts().sort_index()

# Imprimir la distribución de valores de 'chol'
print(distribucion_chol)


chol
0      150
85       1
100      2
117      1
126      1
      ... 
491      1
518      1
529      1
564      1
603      1
Name: count, Length: 203, dtype: Int64


Pasamos a la variable fbs, que evalúa a 1 si hay dolor por esfuerzo o 0 en caso contrario.

In [88]:
# Obtener la distribución de valores de la variable 'fbs'
distribucion_fbs = df['fbs'].value_counts().sort_index()

# Imprimir la distribución de valores de 'fbs'
print(distribucion_fbs)


fbs
-9.0      8
0       138
0.0     415
1        55
1.0      57
?        58
Name: count, dtype: int64


Lo pasamos a entero al no haber flotantes.

In [89]:
# Eliminar los decimales de la columna 'fbs'
df['fbs'] = df['fbs'].astype(str).str.split('.').str[0]

# Obtener la distribución de valores de la variable 'fbs'
distribucion_fbs = df['fbs'].value_counts().sort_index()

# Imprimir la distribución de valores de 'fbs'
print(distribucion_fbs)


fbs
-9      8
0     553
1     112
?      58
Name: count, dtype: int64


Como vemos visualmente, la moda es bastante evidente, el 75% de los pacientes no experimentan dolor por esfuerzo, por lo que los valores anómalos (-9 e ?) los sustituiremos por esta moda, ya que suponemos que lo lógico y común es no sentir dolor, y no son pocos registros como para descartarlos.

In [90]:
# Reemplazar los valores '-9' y '?' por '0' en la columna 'fbs'
df['fbs'].replace(['-9', '?'], '0', inplace=True)

# Convertir la columna 'fbs' a tipo entero
df['fbs'] = df['fbs'].astype(int)

# Obtener la distribución de valores de la variable 'fbs'
distribucion_fbs = df['fbs'].value_counts().sort_index()

# Imprimir la distribución de valores de 'fbs'
print(distribucion_fbs)


fbs
0    619
1    112
Name: count, dtype: int64


Continuamos con la variable restecg, la cual nos indica los resultados electrocardiográficos en reposo.

In [91]:
# Obtener la distribución de valores de la variable 'restecg'
distribucion_restecg = df['restecg'].value_counts()

# Imprimir la distribución de valores de 'restecg'
print(distribucion_restecg)


restecg
0.0    438
2.0    148
1.0    145
Name: count, dtype: int64


Vemos que la disposición de esta variable es correcta, simplemente la pasamos a entero como de costumbre.

In [92]:
# Convertir la columna 'restecg' a tipo entero
df['restecg'] = df['restecg'].astype(int)

# Obtener la distribución de valores de la variable 'restecg'
distribucion_restecg = df['restecg'].value_counts().sort_index()

# Imprimir la distribución de valores de 'restecg'
print(distribucion_restecg)


restecg
0    438
1    145
2    148
Name: count, dtype: int64


Continuamos con thalach, que representa la frecuencia cardiaca en reposo.

In [93]:
# Obtener la distribución de valores de la variable 'thalach' y ordenarla de menor a mayor
distribucion_thalach = df['thalach'].value_counts().sort_index()

# Imprimir la distribución de valores de 'thalach' ordenada de menor a mayor
print(distribucion_thalach)


thalach
100       7
100.0     6
102       3
102.0     1
103       1
         ..
98        4
98.0      3
99        3
99.0      2
?        44
Name: count, Length: 180, dtype: int64


De manera habitual, al no haber decimales los descartamos.

In [94]:
# Eliminar los decimales de la columna 'thalach'
df['thalach'] = df['thalach'].astype(str).str.split('.').str[0]

# Obtener la distribución de valores de la variable 'thalach' y ordenarla de menor a mayor
distribucion_thalach = df['thalach'].value_counts().sort_index()

# Imprimir la distribución de valores de 'thalach' ordenada de menor a mayor
print(distribucion_thalach)


thalach
100    13
102     4
103     4
104     1
105     8
       ..
96      6
97      2
98      7
99      5
?      44
Name: count, Length: 115, dtype: int64


Aquí vemos que la distribución es bastante diversa, y tenemos que 44 registros no están asignados, por lo que debemos darles un valor.

Para ello, veremos a qué tienden los valores.

In [95]:
import numpy as np

# Filtrar los valores diferentes de '?' en la columna 'thalach'
df_filtered = df[df['thalach'] != '?']

# Convertir la columna 'thalach' a tipo numérico
df_filtered['thalach'] = pd.to_numeric(df_filtered['thalach'])

# Calcular la media de los valores en la columna 'thalach'
media_thalach = df_filtered['thalach'].mean()

# Calcular la moda de los valores en la columna 'thalach'
moda_thalach = df_filtered['thalach'].mode()[0]

# Imprimir la media y la moda de 'thalach'
print("Media de thalach:", media_thalach)
print("Moda de thalach:", moda_thalach)


Media de thalach: 138.10771470160117
Moda de thalach: 140


<ipython-input-95-f0eece54ca33>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['thalach'] = pd.to_numeric(df_filtered['thalach'])


Vemos que deja bastante evidente que tanto la media como la moda tienden a que lo más común en estas enfermedades es tener una frecuencia cardiaca en reposo entorno a 140, por lo que otorgaremos este valor a los registros desconocidos.

In [96]:
# Reemplazar los valores '?' por '140' en la columna 'thalach'
df['thalach'].replace('?', '140', inplace=True)

# Mostrar la distribución de valores de la variable 'thalach'
distribucion_thalach = df['thalach'].value_counts().sort_index()
print(distribucion_thalach)


thalach
100    13
102     4
103     4
104     1
105     8
       ..
95      2
96      6
97      2
98      7
99      5
Name: count, Length: 114, dtype: int64


Continuamos con exang, que representa con 1 si existe angina inducida por ejercicio, o 0 en caso contrario

In [97]:
# Mostrar la distribución de valores de la variable 'exang'
distribucion_exang = df['exang'].value_counts()
print(distribucion_exang)


exang
0.0    327
1.0    153
1      113
0       94
?       44
Name: count, dtype: int64


Lo primero como siempre, descartamos decimales al no haber flotantes.

Al haber una clara tendencia hacia una no presencia de angina, asignaremos 0 a los registros con ?.

In [98]:
# Convertir la columna 'exang' a tipo entero
df['exang'] = df['exang'].replace('?', np.nan).astype(float).fillna(0).astype(int)

# Obtener la distribución de valores de la variable 'exang' y ordenarla de menor a mayor
distribucion_exang = df['exang'].value_counts().sort_index()

# Imprimir la distribución de valores de 'exang' ordenada de menor a mayor
print(distribucion_exang)


exang
0    465
1    266
Name: count, dtype: int64


Continuamos con oldpeak, que representa la depresión del ST inducida por el ejercicio en relación con el reposo.

In [99]:
# Obtener la distribución de valores de la variable 'oldpeak' y ordenarla de menor a mayor
distribucion_oldpeak = df['oldpeak'].value_counts().sort_index()

# Imprimir la distribución de valores de 'oldpeak' ordenada de menor a mayor
print(distribucion_oldpeak)


oldpeak
-.5      1
-.8      1
-.9      1
-0.5     1
-1       2
        ..
4.4      1
5.0      1
5.6      1
6.2      1
?       49
Name: count, Length: 64, dtype: int64


Aquí vemos que sí hay decimales, por lo que no interesa descartarlos, y vemos que su distribución es más compleja, por lo que vamos a ver a qué tiende.

In [100]:
# Convertir la columna 'oldpeak' a tipo numérico
df['oldpeak'] = pd.to_numeric(df['oldpeak'], errors='coerce')

# Contar los valores negativos, positivos, cero, nulos e interrogantes en la columna 'oldpeak' considerando los decimales
negativos = (df['oldpeak'] < 0).sum()
positivos = (df['oldpeak'] > 0).sum()
ceros = (df['oldpeak'] == 0).sum()

# Imprimir el conteo de valores negativos, positivos, cero, nulos e interrogantes
print("Valores negativos:", negativos)
print("Valores positivos:", positivos)
print("Valores cero:", ceros)


Valores negativos: 10
Valores positivos: 375
Valores cero: 297


In [101]:
import numpy as np

# Filtrar los valores diferentes de '?' en la columna 'oldpeak'
df_filtered = df[df['oldpeak'] != '?']

# Convertir la columna 'oldpeak' a tipo numérico
df_filtered['oldpeak'] = pd.to_numeric(df_filtered['oldpeak'], errors='coerce')

# Calcular la media de los valores en la columna 'oldpeak'
media_oldpeak = df_filtered['oldpeak'].mean()

# Calcular la moda de los valores en la columna 'oldpeak'
moda_oldpeak = df_filtered['oldpeak'].mode()[0]

# Imprimir la media y la moda de 'oldpeak'
print("Media de oldpeak:", media_oldpeak)
print("Moda de oldpeak:", moda_oldpeak)


Media de oldpeak: 0.8803519061583578
Moda de oldpeak: 0.0


Vemos que existen valores negativos que no tienen por qué ser anómalos, por lo que de momento no los vamos a descartar. Vemos que tanto la media como la moda tienden a que el denominador más común suele ser 0, por lo que los 49 registros desconocidos los evaluaremos a 0.

In [102]:
# Reemplazar los valores '?' por 0.0 en la columna 'oldpeak'
df['oldpeak'].replace('?', '0.0', inplace=True)

# Mostrar la distribución de valores de la columna 'oldpeak'
distribucion_oldpeak = df['oldpeak'].value_counts().sort_index()
print(distribucion_oldpeak)


oldpeak
-2.6      1
-2.0      1
-1.5      1
-1.1      1
-1.0      2
-0.9      1
-0.8      1
-0.5      2
 0.0    297
 0.1      7
 0.2     10
 0.3      5
 0.4      8
 0.5     17
 0.6     11
 0.7      2
 0.8     11
 0.9      4
 1.0     65
 1.1      3
 1.2     11
 1.3      4
 1.4     12
 1.5     34
 1.6     12
 1.7      2
 1.8     10
 1.9      4
 2.0     60
 2.1      2
 2.2      5
 2.3      1
 2.4      3
 2.5     13
 2.6      7
 2.8      5
 2.9      1
 3.0     22
 3.1      1
 3.2      2
 3.4      3
 3.5      2
 3.6      2
 3.7      1
 3.8      1
 4.0      6
 4.2      2
 4.4      1
 5.0      1
 5.6      1
 6.2      1
Name: count, dtype: int64


Continuamos con slope, que representa la pendiente del segmento ST en ejercicio máximo, que solo puede tomar los valores 1, 2 y 3 (pendiente ascendente, plano y descendente respectivamente)

In [103]:
# Mostrar la distribución de valores de la variable 'slope'
distribucion_slope = df['slope'].value_counts().sort_index()
print(distribucion_slope)


slope
-9.0    152
1        37
1.0     121
2        89
2.0     187
3        30
3.0      20
?        95
Name: count, dtype: int64


Aquí sí que lo primero es eliminar los decimales al no haber flotantes.

In [104]:
# Convertir la columna 'slope' a tipo string y eliminar decimales
df['slope'] = df['slope'].astype(str).str.split('.').str[0]

# Mostrar la distribución de valores de la variable 'slope'
distribucion_slope = df['slope'].value_counts().sort_index()
print(distribucion_slope)


slope
-9    152
1     158
2     276
3      50
?      95
Name: count, dtype: int64


Aquí al no conocer el alcance exacto de la variable ni la importancia, y al haber una gran cantidad de registros desconocidos que no podemos descartar al ser muy elevado, procedemos a cambiarlos por la moda, que parece a priori que tiene una tendencia plana.

In [107]:
# Reemplazar los valores '?' por 1 en la columna 'slope'
df['slope'].replace('?', '1', inplace=True)

# Reemplazar los valores -9 por 1 en la columna 'slope'
df['slope'].replace('-9', '1', inplace=True)

# Convertir la columna 'slope' a tipo entero
df['slope'] = df['slope'].astype(int)

# Mostrar la distribución actualizada de valores de la variable 'slope'
distribucion_slope = df['slope'].value_counts().sort_index()
print(distribucion_slope)


slope
1    405
2    276
3     50
Name: count, dtype: int64


Pasamos a la variable ca, que corresponde con el número de vasos mayores (0-3) coloreados por flouroscopia.

In [108]:
# Mostrar la distribución de valores de la variable 'ca'
distribucion_ca = df['ca'].value_counts().sort_index()
print(distribucion_ca)


ca
-9.0    230
0         2
0.0     145
1         1
1.0      53
2         2
2.0      33
3.0      17
?       248
Name: count, dtype: int64


En primer lugar, como siempre, descartamos decimales al no existir flotantes.

In [109]:
# Descartar los decimales en la distribución de valores de la variable 'ca'
df['ca'] = df['ca'].astype(str).str.split('.').str[0]

# Mostrar la distribución actualizada de valores de la variable 'ca'
distribucion_ca = df['ca'].value_counts().sort_index()
print(distribucion_ca)


ca
-9    230
0     147
1      54
2      35
3      17
?     248
Name: count, dtype: int64


Vemos que a priori no va a resultar una variable muy directa en el aprendizaje de la enfermedad, ya que hay más valores desconocidos que conocidos, por lo que simplemente los sustituiremos por la moda, que es el 0.

In [110]:
# Reemplazar los valores '?' por '0' en la columna 'ca'
df['ca'].replace('?', '0', inplace=True)

# Reemplazar los valores '-9' por '0' en la columna 'ca'
df['ca'].replace('-9', '0', inplace=True)

# Mostrar la distribución actualizada de valores de la variable 'ca'
distribucion_ca = df['ca'].value_counts().sort_index()
print(distribucion_ca)


ca
0    625
1     54
2     35
3     17
Name: count, dtype: int64


Para finalizar la limpieza, analizaremos la variable thal, que representa con los valores 3, 6 y 7 los valores normal, defecto fijo y defecto reversible respectivamente.

In [111]:
# Mostrar la distribución de valores de la variable 'thal'
distribucion_thal = df['thal'].value_counts().sort_index()
print(distribucion_thal)


thal
-9.0    210
3        20
3.0     137
6        14
6.0      22
7        50
7.0     110
?       168
Name: count, dtype: int64


Descartamos decimales de la manera habitual.

In [112]:
# Descartar los decimales en la distribución de valores de la variable 'thal'
df['thal'] = df['thal'].astype(str).str.split('.').str[0]

# Mostrar la distribución actualizada de valores de la variable 'thal'
distribucion_thal = df['thal'].value_counts().sort_index()
print(distribucion_thal)


thal
-9    210
3     157
6      36
7     160
?     168
Name: count, dtype: int64


Aquí, de manera similar a la varible ca estudiada previamente, al existir multitud de registros desconocidos y no sacar una tendencia clara, decidimos otorgar a los desconocidos un thal normal, de cara a alterar el algoritmo lo menos posible.

In [113]:
# Reemplazar los valores '?' por '3' en la columna 'thal'
df['thal'].replace('?', '3', inplace=True)

# Reemplazar los valores '-9' por '3' en la columna 'thal'
df['thal'].replace('-9', '3', inplace=True)

# Mostrar la distribución actualizada de valores de la variable 'thal'
distribucion_thal = df['thal'].value_counts().sort_index()
print(distribucion_thal)


thal
3    535
6     36
7    160
Name: count, dtype: int64


Con esto daríamos por finalizada la primera limpieza del dataset de entrenamiento, por lo que podemos proceder a descargarlo.

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 731 entries, 0 to 731
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       731 non-null    int64  
 1   sex       731 non-null    int64  
 2   cp        731 non-null    int64  
 3   trestbps  731 non-null    int64  
 4   chol      726 non-null    Int64  
 5   fbs       731 non-null    int64  
 6   restecg   731 non-null    int64  
 7   thalach   731 non-null    object 
 8   exang     731 non-null    int64  
 9   oldpeak   682 non-null    float64
 10  slope     731 non-null    int64  
 11  ca        731 non-null    object 
 12  thal      731 non-null    object 
 13  label     731 non-null    int64  
dtypes: Int64(1), float64(1), int64(9), object(3)
memory usage: 86.4+ KB


In [114]:
# Guardar el DataFrame modificado como un nuevo archivo CSV
df.to_csv('train_cleaned.csv', index=False)